# Generate EMA using dedicated service - C#

### Overview
Moving averages smooth the price movements by filtering out the "noise" caused by short-term price fluctuations.They also form the building blocks for a wide range of technical indicators.

Computing moving averages over a defined number of time periods is useful to:
* identify the trend direction
* determine support and resistance levels

This sample demonstrates how to request and plot **on-demand** *exponential moving average - EMA* from a dedicated data service.

### Inputs/outputs
Exponential moving averages' requests require instrument's identifier, date time intervals and number of points as per inputs.

EMA reduces the lag by applying greater weight to recent data points.The weighting applied to the most recent price depends on the number of points in the moving average; The smaller the point count is, the more weight is applied to the most recent price.

N-points EMA data point is computed using the previous' *EMA* value and the wighting multiplier as follows:

$$ EMA = (Price-EMAp)*k+EMAp $$

Where:
* *Price* is the trade price
* *EMAp* is the previous points' *EMA* value, *SMA* is used as the EMAp in the first calculation
* *N* is the count of points
* *$ k = \frac{2}{N+1} $* is the *Weighting Multiplier*

<div class="alert alert-block alert-info">
    <b>Note:</b> <i>EMA</i> adapts quickly to price changes than the <i>SMA</i>, when a price reverses direction, the <i>EMA</i> will reverse direction quicker than the <i>SMA</i> dur to the weighting 
</div>

This sample shows how to plot a simple graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve EMA indicator from the hosted service. The queried endpoint in this script are:
* *TickEmaService*: to directly retrieve EMA data from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.TickAnalytics.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run EMA sample

### Step 1: Install packages

In [ ]:
#r "nuget: Systemathics.Apis"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.TickAnalytics.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Core;
using Grpc.Net.Client;
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Retrieve authentication token
var token = System.Environment.GetEnvironmentVariable("AUTH0_TOKEN");

// Add token to the request header
var headers = new Metadata();
headers.Add("Authorization", $"Bearer {token}");

// Define API entry
var channel = GrpcChannel.ForAddress("https://apis.systemathics.cloud");

In [ ]:
// Display authentication token 
display(token);

### Step 3: Create and process request
To request *EMA* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Exponential moving average request parameters

#### 3.1 Instrument selection

In [ ]:
var ticker = "AAPL";
var exchange = "BATS";

#### 3.2 EMA parameters

In [ ]:
var field = EmaPrice.Trade;
var longEma = 200;
var shortEma = 50;
var sampling = new Duration { Seconds = 1 * 60 };

#### 3.2 Time period delimitation

In [ ]:
var today = DateTime.Today;
var dateIntervals = new DateInterval
{
    StartDate = new Date { Year = 2020, Month = 02, Day = 18 },
    EndDate = new Date { Year = 2020, Month = 02, Day = 18 }
};

var timeInterval = new TimeInterval
{
    StartTime = new TimeOfDay { Hours = 08, Minutes = 00, Seconds = 00 },
    EndTime = new TimeOfDay { Hours = 20, Minutes = 00, Seconds = 00 }
};

#### 3.3 Request creation
The following code snippets creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate constraints based on the previous space selection:
var constraints = new Constraints();
constraints.DateIntervals.Add(dateIntervals);
constraints.TimeIntervals.Add(timeInterval);   

The following code snippets create requests for: *Long EMA*, *Short EMA* and *Prices*:

In [ ]:
// Generate 200-points EMA request
var emaLongRequest = new TickEmaRequest()
{
    Identifier = new Identifier { Exchange = exchange, Ticker = ticker}, 
    Constraints = constraints,  
    Field = field,
    Length = longEma, 
    Sampling = sampling
};

// Generate 50-points EMA request
var emaShortRequest = new TickEmaRequest()
{
    Identifier = new Identifier { Exchange = exchange, Ticker = ticker}, 
    Constraints = constraints,  
    Field = field,
    Length = shortEma, 
    Sampling = sampling
};

// Instantiate the tick EMA service
var service = new TickEmaService.TickEmaServiceClient(channel);

### Step 4: Visualize data

#### 4.1 Retrieve long EMA data

In [ ]:
var emalong = new List<TickEmaResponse>();

var call = service.TickEma(emaLongRequest, headers);
await foreach (var data in call.ResponseStream.ReadAllAsync())
{
    emalong.Add(data);
}
//display(emalong);

#### 4.2 Retrieve short EMA data

In [ ]:
var emashort = new List<TickEmaResponse>();

var call = service.TickEma(emaShortRequest, headers);
await foreach (var data in call.ResponseStream.ReadAllAsync())
{
    emashort.Add(data);
}
//display(emashort);

#### 4.1 Plot EMA and prices
Plot the EMA and prices request' results with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package. Plot a simple line graph with the EMA and prices data points: 

In [ ]:
var emaPlots = new List<Scattergl>
{
    new Scattergl
    {
        name = "200-points EMA",
        mode = "lines",
        x = emalong.Select(l => l.TimeStamp.ToDateTime()),
        y = emalong.Select(l => l.Average)
    },
    new Scattergl
    {
        name = "50-points EMA",
        mode = "lines",
        x = emashort.Select(l => l.TimeStamp.ToDateTime()),
        y = emashort.Select(l => l.Average)
    },
    new Scattergl
    {
        name = "Prices",
        mode = "lines",
        x = emalong.Select(l => l.TimeStamp.ToDateTime()),
        y = emalong.Select(l => l.Value)
    }
};
var chart = Chart.Plot(emaPlots);
chart.WithTitle($"EMAs and prices for {ticker}");
chart.WithXTitle($"Time");
chart.WithYTitle($"Price");
chart.Width = 1500;
chart.Height = 800;
display(chart);